In [1]:
# 1. Take video inputs
# 2. Extract last frame of the video
# 3. Implement yolo detector.
# 4. Pass 4 frames to detector.
# 5. Get detection of each frame class wise.
# 6. Convert large vehicle count to small vehicels equivalant.
# 7. Aggregate counts of vehicles for each lanes.
# 8. Pass counts to timer algo.
# 9. Get results of timer algo.
# 10. Print timer to be set for next green signal.
# STOP


# New
# 0. Load google drive.
# 1. Load Code and data from google drive
# 2. Install YOLO dependencies
# 3. Select videos.
# 4. Extract frames.
# 5. Import YOLO and timer modules.
# 6. Detect objects in frames.
# 7. Get counts from the detections.
# 8. Convert to density.
# 9. Pass to timing algorithm.
# 10. Print results.

# Traffic Controller
## Backend.

Flow:

Video Input -> Object detection -> timing algorithm -> Timer outputs.

### Load Videos
Import testing videos from Google Drive.

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir test_videos

dest = "./test_videos"
for i in range(1, 10):
    source = f"./drive/MyDrive/Main\ Project/Project\ Data\ [MISC]/Code/Data/Test\ videos/Video{i}.mp4"
    !cp {source} {dest}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘test_videos’: File exists


### Set which videos to load

In [3]:
input_videos = [f"./test_videos/Video{i}.mp4" for i in (2, 3, 7)]

### Load Selected videos using openCV

In [4]:
import numpy as np
import cv2 as cv

# Use this if showing video frames...
# from google.colab.patches import cv2_imshow

# Load all 4 videos of each lane.
caps = []
for video_path in input_videos:
    caps.append( cv.VideoCapture(video_path) )

frames = []
for cap in caps:
    ret, frame = cap.read()
    if ret:
        frames.append(frame)
    else:
        print("Some video not loading properly.")

### Resize the frames
Resize all frames to 416x416 size.

In [5]:
for i in range(len(frames)):
    frames[i] = cv.resize(frames[i], (416, 416))

### Install dependencies for YOLO


In [6]:
! pip install -q -r ./yolo/requirements-yolov3.txt

### Load YOLO object detection model.

In [7]:
import importlib
from yolo import yolov3
# importlib.reload(yolov3)

Ready!


### Detect frames

In [8]:
result = yolov3.detect_multiple(frames)

### Extract counts from each detections.

In [9]:
counts = []
for i, res in enumerate(result):
    print(f"Frame {i+1}: ")
    count = {}
    for key in res:
        print("   ", key, res[key][0])
        count[key] = res[key][0]

    counts.append(count)

Frame 1: 
    car 12
    bus 2
    truck 3
Frame 2: 
    car 14
    bus 1
    truck 4
Frame 3: 
    car 13
    bike 4


### Normalize the Counts

In [10]:
# For each frame get grand total of vehicles
total_counts_per_lane = []

conversion_dict = {
    'truck': lambda x: x*5,
    'bus': lambda x: x*4,
    'car': lambda x: x*2,
    'bike': lambda x: x
}

for count_dict in counts:
    total_count = 0
    for vType, count in count_dict.items():
        total_count += conversion_dict[ vType ] ( count )

    total_counts_per_lane.append(total_count)

total_counts_per_lane

[47, 52, 30]

### Load Timer algorithm

In [11]:
# import importlib
from timer import Timer
# importlib.reload(Timer)

In [12]:
timer_1 = Timer()
timer_1.analyze(total_counts_per_lane[:3])

Threshold value 43.0
Updated Classes {'L1': 'M', 'L2': 'M', 'L3': 'L'}
Updated timers  {'L1': 45, 'L2': 45, 'L3': 30}

▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒



In [13]:
for i in range(len(total_counts_per_lane) ):
    print(f"{i+1} {total_counts_per_lane[i]}")

1 47
2 52
3 30
